In [3]:
import pyspark
import pandas as pd
from textblob import TextBlob
from pyspark.sql.functions import udf
import findspark

In [3]:
def get_adjectives(text):
    blob = TextBlob(text)
    return [ word for (word,tag) in blob.tags if tag == "JJ"]

In [4]:
pd.read_csv(r'C:\Users\nitis\OneDrive\Dokumente\movie_sentiment\aftercorona.csv')

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,Damn I thought I was finally in the #Matrix or...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets ...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black M...,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,The perfect description of the #Tories and the...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\n\nONEUS is on now s...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,1.243260e+18,600,CatheGrisson,It's getting harder every day to be a fan of @...,0,"Las Vegas, NV",17/06/2021,146,False,2,NaN,#thewalkingdead,en,date post corona\2021-06-18_en_#thewalkingdead...
1731,2.552374e+08,26940,DavidOpie,#BlackSummer season 2 is out today on @Netflix...,0,"London, England",17/06/2021,5467,False,0,NaN,#thewalkingdead,en,date post corona\2021-06-18_en_#thewalkingdead...
1732,1.063830e+18,1932,Alicia05021,Are we even getting a trailer or are we just g...,0,NaN,17/06/2021,83,False,0,NaN,#thewalkingdead,en,date post corona\2021-06-18_en_#thewalkingdead...
1733,1.393000e+18,315,onterrorstreet,#TheWalkingDead Final Season Premiere Reveal B...,1,NaN,17/06/2021,220,False,9,NaN,#thewalkingdead,en,date post corona\2021-06-18_en_#thewalkingdead...


In [5]:
#creating a spark session
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('practice').getOrCreate()

In [7]:
spark

In [8]:
#read twitter csv file
df_pyspark = spark.read.option('header','true').csv(r'C:\Users\nitis\OneDrive\Dokumente\movie_sentiment\aftercorona.csv', sep=',', multiLine=True)
df_pyspark
# here it shows all the column and the data type

DataFrame[user_id: string, user_statuses: string, name: string, tweet: string, retweets: string, location: string, created: string, followers: string, is_user_verified: string, favorite_count: string, in_reply_to_status_id: string, keyword: string, language: string, site: string]

In [9]:
# see the dataframe in spark
df_pyspark.show()

+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+----------+---------+----------------+--------------+---------------------+------------+--------------------+--------------------+
|             user_id| user_statuses|                name|               tweet|            retweets|            location|   created|followers|is_user_verified|favorite_count|in_reply_to_status_id|     keyword|            language|                site|
+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+----------+---------+----------------+--------------+---------------------+------------+--------------------+--------------------+
|            27307763|         50931|         AppelquistP|Damn I thought I ...|                   0|                  OZ|18/06/2021|      130|           FALSE|             0|                 null|#blackmirror|                  en|date post coro

In [10]:
#now the dataframe is spark dataframe and pandas
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [11]:
#here the head works diffrently too
df_pyspark.head(3)

[Row(user_id='27307763', user_statuses='50931', name='AppelquistP', tweet='Damn I thought I was finally in the #Matrix or #blackmirror or #Aliens landed and this was their way of a soft opening...\nNow, I appreciate #Interns even more.. https://t.co/AjdTC8V6tq', retweets='0', location='OZ', created='18/06/2021', followers='130', is_user_verified='FALSE', favorite_count='0', in_reply_to_status_id=None, keyword='#blackmirror', language='en', site='date post corona\\2021-06-18_en_#blackmirror.json'),
 Row(user_id='2880588386', user_statuses='27578', name='kaarnama13', tweet='â€œNosediveâ€\x9d episode from #blackmirror gets closer to reality with kencil culture', retweets='0', location='Aage se right', created='18/06/2021', followers='669', is_user_verified='FALSE', favorite_count='6', in_reply_to_status_id=None, keyword='#blackmirror', language='en', site='date post corona\\2021-06-18_en_#blackmirror.json'),
 Row(user_id='2370116357', user_statuses='115767', name='Niggvtivity', tweet="I'v

In [12]:
# like df.info in pandas
df_pyspark.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_statuses: string (nullable = true)
 |-- name: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- retweets: string (nullable = true)
 |-- location: string (nullable = true)
 |-- created: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- is_user_verified: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- language: string (nullable = true)
 |-- site: string (nullable = true)



In [13]:
df_pyspark = df_pyspark.select(['tweet','keyword'])
df_pyspark.show()

+--------------------+------------+
|               tweet|     keyword|
+--------------------+------------+
|Damn I thought I ...|#blackmirror|
|â€œNosediveâ€ ep...|#blackmirror|
|I've just watched...|#blackmirror|
|The perfect descr...|#blackmirror|
|ã€ #ONEUS Ã— #sm...|#blackmirror|
|@BBN_Ireland @Fat...|#blackmirror|
|Imagining if TikT...|#blackmirror|
|@ArshadParvez Nei...|#blackmirror|
|#UK #Canada #Fran...|#blackmirror|
|"I have been watc...|#blackmirror|
|@netflix Please I...|#blackmirror|
|So I've finally g...|#blackmirror|
|@lockwoodx3 It is...|#blackmirror|
|Model X will be l...|#blackmirror|
|Whatâ€™s your fav...|#blackmirror|
|@ButterHoops Same...|#blackmirror|
|Oh! And #blackmirror|#blackmirror|
|INSIDE BLACK MIRR...|#blackmirror|
|"PARALLEL UNIVERS...|        null|
|                   1|          en|
+--------------------+------------+
only showing top 20 rows



In [14]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [16]:
# to read custom function, we need to import udf which is user defined function
example_udf = udf(get_adjectives)
df_pyspark = df_pyspark.withColumn('result', example_udf(df_pyspark.tweet))
df_pyspark.show()

Py4JJavaError: An error occurred while calling o79.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (10.90.0.118 executor driver): java.io.IOException: Cannot run program "C:\Users\nitis\anaconda3": CreateProcess error=5, Access is denied
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$a